Generalised Additive Model with MICE and Target Encoding

In [2]:
import os
os.chdir('../')

In [3]:
import numpy as np
import pandas as pd

data=pd.read_csv('Datasets/analysis_data.csv')

In [6]:
from sklearn.model_selection import train_test_split, KFold
from sklearn.experimental import enable_iterative_imputer  # noqa: F401
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge
from sklearn.metrics import mean_squared_error, root_mean_squared_error

from pygam import LinearGAM, s  # Generalized Additive Model

# =========================================================
# 1. Setup: target and features
# =========================================================
df = data.copy()  # assumes `data` exists
target_col = "monthly_spend"

y = df[target_col].reset_index(drop=True)
X = df.drop(columns=[target_col]).reset_index(drop=True)

# Identify column types
cat_cols = X.select_dtypes(include=["object", "category"]).columns.tolist()
num_cols = X.select_dtypes(include=["number"]).columns.tolist()

print("Categorical columns:", cat_cols)
print("Numeric columns:", num_cols)

# =========================================================
# 2. Train–test split
# =========================================================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Work on copies
X_train_enc = X_train.copy().reset_index(drop=True)
X_test_enc = X_test.copy().reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

# Ensure categoricals are strings and handle missing
if cat_cols:
    X_train_enc[cat_cols] = X_train_enc[cat_cols].astype(str).fillna("Missing")
    X_test_enc[cat_cols] = X_test_enc[cat_cols].astype(str).fillna("Missing")

# =========================================================
# 3. Out-of-fold Target Encoding (LEAK-PROOF)
# =========================================================
def target_encode_train_test(
    X_train_df, y_train_ser, X_test_df, cat_columns, n_splits=5, smoothing=10
):
    """
    Returns:
        X_train_te: DataFrame with target-encoded train columns
        X_test_te:  DataFrame with target-encoded test columns
    """
    X_train_te = pd.DataFrame(index=X_train_df.index)
    X_test_te = pd.DataFrame(index=X_test_df.index)

    if not cat_columns:
        return X_train_te, X_test_te

    y_train_ser = y_train_ser.reset_index(drop=True)
    global_mean = y_train_ser.mean()

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

    # ----- TRAIN (out-of-fold) -----
    for col in cat_columns:
        oof = pd.Series(index=X_train_df.index, dtype=float)

        for tr_idx, val_idx in kf.split(X_train_df):
            X_tr = X_train_df.iloc[tr_idx]
            X_val = X_train_df.iloc[val_idx]
            y_tr = y_train_ser.iloc[tr_idx]

            stats = (
                y_tr.groupby(X_tr[col])
                .agg(["mean", "count"])
                .rename(columns={"mean": "te_mean", "count": "te_count"})
            )

            smoothing_factor = 1 / (1 + np.exp(-(stats["te_count"] - smoothing)))
            te_values = global_mean * (1 - smoothing_factor) + stats["te_mean"] * smoothing_factor

            oof.iloc[val_idx] = X_val[col].map(te_values)

        oof = oof.fillna(global_mean)
        X_train_te[col + "_te"] = oof

    # ----- TEST (full-data encoding) -----
    for col in cat_columns:
        stats_full = (
            y_train_ser.groupby(X_train_df[col])
            .agg(["mean", "count"])
            .rename(columns={"mean": "te_mean", "count": "te_count"})
        )

        smoothing_factor_full = 1 / (1 + np.exp(-(stats_full["te_count"] - smoothing)))
        te_values_full = global_mean * (1 - smoothing_factor_full) + stats_full["te_mean"] * smoothing_factor_full

        test_encoded = X_test_df[col].map(te_values_full).fillna(global_mean)
        X_test_te[col + "_te"] = test_encoded

    return X_train_te, X_test_te

X_train_te, X_test_te = target_encode_train_test(
    X_train_enc, y_train, X_test_enc, cat_cols, n_splits=5, smoothing=10
)

# =========================================================
# 4. Combine numeric + target-encoded categorical features
# =========================================================
X_train_num = X_train_enc[num_cols].copy()
X_test_num = X_test_enc[num_cols].copy()

X_train_final = pd.concat(
    [X_train_num.reset_index(drop=True),
     X_train_te.reset_index(drop=True)],
    axis=1
)
X_test_final = pd.concat(
    [X_test_num.reset_index(drop=True),
     X_test_te.reset_index(drop=True)],
    axis=1
)

print("Train shape before MICE:", X_train_final.shape)
print("Test shape before MICE:", X_test_final.shape)

# =========================================================
# 5. MICE Imputation (IterativeImputer with BayesianRidge)
# =========================================================
mice = IterativeImputer(
    estimator=BayesianRidge(),
    max_iter=10,
    initial_strategy="median",
    random_state=42
)

X_train_imputed = mice.fit_transform(X_train_final)
X_test_imputed = mice.transform(X_test_final)

X_train_imputed = pd.DataFrame(X_train_imputed, columns=X_train_final.columns)
X_test_imputed = pd.DataFrame(X_test_imputed, columns=X_test_final.columns)

print("Any NaNs after MICE (train)?", X_train_imputed.isna().sum().sum())
print("Any NaNs after MICE (test)? ", X_test_imputed.isna().sum().sum())

# =========================================================
# 6. Fit a GAM (LinearGAM) on imputed numeric + TE features
# =========================================================
# We’ll use one smooth term s(...) per feature.
# You can tune lam, n_splines, etc., for better performance.

n_features = X_train_imputed.shape[1]
terms = sum([s(i) for i in range(n_features)], start=s(0))  # additive smooth terms

gam = LinearGAM(terms).gridsearch(
    X_train_imputed.values,
    y_train.values,
    lam=[0.1, 1, 10]  # smoothing penalties to try
)

# =========================================================
# 7. Evaluate on test set
# =========================================================
y_pred_test = gam.predict(X_test_imputed.values)
rmse_gam = root_mean_squared_error(y_test, y_pred_test)
print(f"\n🔥 GAM + Target Encoding + MICE — Test RMSE: {rmse_gam:.4f}")

# Optional: check R² as well
from sklearn.metrics import r2_score
r2_gam = r2_score(y_test, y_pred_test)
print(f"GAM Test R²: {r2_gam:.4f}")

# =========================================================
# 8. Inspect partial dependence (shape functions) for key features
# =========================================================
# Example: plot shapes for first few features (requires matplotlib)
import matplotlib.pyplot as plt

feature_names = X_train_imputed.columns.tolist()




Categorical columns: ['gender', 'marital_status', 'education_level', 'region', 'employment_status', 'card_type']
Numeric columns: ['customer_id', 'age', 'owns_home', 'has_auto_loan', 'annual_income', 'credit_score', 'credit_limit', 'tenure', 'num_transactions', 'avg_transaction_value', 'online_shopping_freq', 'reward_points_balance', 'travel_frequency', 'utility_payment_count', 'num_children', 'num_credit_cards']
Train shape before MICE: (32000, 22)
Test shape before MICE: (8000, 22)


  0% (0 of 3) |                          | Elapsed Time: 0:00:00 ETA:  --:--:--


Any NaNs after MICE (train)? 0
Any NaNs after MICE (test)?  0


 33% (1 of 3) |########                  | Elapsed Time: 0:00:07 ETA:   0:00:15
 66% (2 of 3) |#################         | Elapsed Time: 0:00:14 ETA:   0:00:07
100% (3 of 3) |##########################| Elapsed Time: 0:00:21 Time:  0:00:21



🔥 GAM + Target Encoding + MICE — Test RMSE: 258.4614
GAM Test R²: 0.7684


Gam without Target Encoding

In [7]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.experimental import enable_iterative_imputer  # noqa: F401
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge
from sklearn.metrics import root_mean_squared_error, r2_score

from pygam import LinearGAM, s


# =========================================================
# 1. SETUP — Split data
# =========================================================
df = data.copy()
target_col = "monthly_spend"

y = df[target_col].reset_index(drop=True)
X = df.drop(columns=[target_col]).reset_index(drop=True)

cat_cols = X.select_dtypes(include=["object", "category"]).columns.tolist()
num_cols = X.select_dtypes(include=["number"]).columns.tolist()

print("Categoricals:", cat_cols)
print("Numerics:", num_cols)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


# =========================================================
# 2. OneHotEncode categorical variables
# =========================================================
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), cat_cols)
    ],
    remainder="passthrough"
)

X_train_ohe = preprocessor.fit_transform(X_train)
X_test_ohe = preprocessor.transform(X_test)

# Column names
ohe_cols = preprocessor.named_transformers_["cat"].get_feature_names_out(cat_cols)
final_cols = list(ohe_cols) + num_cols

X_train_df = pd.DataFrame(X_train_ohe, columns=final_cols)
X_test_df = pd.DataFrame(X_test_ohe, columns=final_cols)

print("Train shape after OHE:", X_train_df.shape)


# =========================================================
# 3. MICE Imputation
# =========================================================
mice = IterativeImputer(
    estimator=BayesianRidge(),
    max_iter=10,
    initial_strategy="median",
    random_state=42
)

X_train_imp = mice.fit_transform(X_train_df)
X_test_imp = mice.transform(X_test_df)

X_train_imp = pd.DataFrame(X_train_imp, columns=final_cols)
X_test_imp = pd.DataFrame(X_test_imp, columns=final_cols)


# =========================================================
# 4. Fit GAM
# =========================================================
n_features = X_train_imp.shape[1]

# Build additive smooth terms
terms = s(0)
for i in range(1, n_features):
    terms = terms + s(i)

gam = LinearGAM(terms).gridsearch(
    X_train_imp.values,
    y_train.values,
    lam=np.logspace(-3, 3, 10)
)


# =========================================================
# 5. Evaluate GAM
# =========================================================
y_pred_test = gam.predict(X_test_imp.values)

rmse = root_mean_squared_error(y_test, y_pred_test)
r2 = r2_score(y_test, y_pred_test)

print(f"\n🔥 GAM Test RMSE: {rmse:.4f}")
print(f"📈 GAM Test R²: {r2:.4f}")


Categoricals: ['gender', 'marital_status', 'education_level', 'region', 'employment_status', 'card_type']
Numerics: ['customer_id', 'age', 'owns_home', 'has_auto_loan', 'annual_income', 'credit_score', 'credit_limit', 'tenure', 'num_transactions', 'avg_transaction_value', 'online_shopping_freq', 'reward_points_balance', 'travel_frequency', 'utility_payment_count', 'num_children', 'num_credit_cards']
Train shape after OHE: (32000, 35)


  0% (0 of 10) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--
 10% (1 of 10) |##                       | Elapsed Time: 0:00:12 ETA:   0:01:55
 20% (2 of 10) |#####                    | Elapsed Time: 0:00:25 ETA:   0:01:41
 30% (3 of 10) |#######                  | Elapsed Time: 0:00:38 ETA:   0:01:28
 40% (4 of 10) |##########               | Elapsed Time: 0:00:51 ETA:   0:01:16
 50% (5 of 10) |############             | Elapsed Time: 0:01:04 ETA:   0:01:04
 60% (6 of 10) |###############          | Elapsed Time: 0:01:17 ETA:   0:00:51
 70% (7 of 10) |#################        | Elapsed Time: 0:01:29 ETA:   0:00:38
 80% (8 of 10) |####################     | Elapsed Time: 0:01:42 ETA:   0:00:25
 90% (9 of 10) |######################   | Elapsed Time: 0:01:56 ETA:   0:00:12
100% (10 of 10) |########################| Elapsed Time: 0:02:09 Time:  0:02:09



🔥 GAM Test RMSE: 258.3478
📈 GAM Test R²: 0.7686


GAM with Interactions

In [9]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge
from sklearn.metrics import root_mean_squared_error, r2_score

from pygam import LinearGAM, s, te

# =========================================
# 1. Split data
# =========================================
df = data.copy()
target_col = "monthly_spend"

y = df[target_col].reset_index(drop=True)
X = df.drop(columns=[target_col]).reset_index(drop=True)

cat_cols = X.select_dtypes(include=["object", "category"]).columns.tolist()
num_cols = X.select_dtypes(include=["number"]).columns.tolist()

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# =========================================
# 2. One-Hot Encode categoricals
# =========================================
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), cat_cols)
    ],
    remainder="passthrough"
)

X_train_ohe = preprocessor.fit_transform(X_train)
X_test_ohe = preprocessor.transform(X_test)

ohe_cols = preprocessor.named_transformers_["cat"].get_feature_names_out(cat_cols)
final_cols = list(ohe_cols) + num_cols

X_train_df = pd.DataFrame(X_train_ohe, columns=final_cols)
X_test_df = pd.DataFrame(X_test_ohe, columns=final_cols)

# =========================================
# 3. MICE Imputation
# =========================================
mice = IterativeImputer(
    estimator=BayesianRidge(),
    max_iter=10,
    initial_strategy="median",
    random_state=42
)

X_train_imp = mice.fit_transform(X_train_df)
X_test_imp = mice.transform(X_test_df)

X_train_imp = pd.DataFrame(X_train_imp, columns=final_cols)
X_test_imp = pd.DataFrame(X_test_imp, columns=final_cols)

# =========================================
# 4. Pick top-k features for interactions
#    (based on absolute correlation with y)
# =========================================
corrs = []
for col in final_cols:
    c = np.corrcoef(X_train_imp[col], y_train)[0, 1]
    if np.isnan(c):
        c = 0.0
    corrs.append(abs(c))

corr_series = pd.Series(corrs, index=final_cols).sort_values(ascending=False)

k = 10  # you can try 10, 15, etc.
topk_features = corr_series.head(k).index.tolist()
print("Top-k features for interactions:", topk_features)

# Map feature names to indices
feat_to_idx = {name: i for i, name in enumerate(final_cols)}
topk_indices = [feat_to_idx[f] for f in topk_features]

# =========================================
# 5. Build GAM terms: all main effects + interactions among top-k
# =========================================
n_features = X_train_imp.shape[1]

# Main smooth terms for ALL features
terms = s(0)
for i in range(1, n_features):
    terms = terms + s(i)

# Add tensor interaction terms ONLY for pairs among top-k
for i_idx in range(len(topk_indices)):
    for j_idx in range(i_idx + 1, len(topk_indices)):
        fi = topk_indices[i_idx]
        fj = topk_indices[j_idx]
        terms = terms + te(fi, fj)

# =========================================
# 6. Fit GAM with interactions (GA²M-lite)
# =========================================
gam = LinearGAM(terms).gridsearch(
    X_train_imp.values,
    y_train.values,
    lam=np.logspace(-3, 3, 7)
)

print("\nSelected Lambda(s):", gam.lam)

# =========================================
# 7. Evaluate on test set
# =========================================
y_pred_test = gam.predict(X_test_imp.values)

rmse = root_mean_squared_error(y_test, y_pred_test)
r2 = r2_score(y_test, y_pred_test)

print(f"\n🔥 GAM (all main effects + interactions among top-{k}) — Test RMSE: {rmse:.4f}")
print(f"📈 GAM (top-{k} interactions) — Test R²: {r2:.4f}")


  0% (0 of 7) |                          | Elapsed Time: 0:00:00 ETA:  --:--:--


Top-k features for interactions: ['credit_limit', 'annual_income', 'reward_points_balance', 'num_transactions', 'travel_frequency', 'card_type_platinum', 'online_shopping_freq', 'card_type_standard', 'utility_payment_count', 'num_children']


c:\Users\athar\anaconda4\Lib\site-packages\pygam\pygam.py:806: RuntimeWarning: divide by zero encountered in scalar divide
  diff = np.linalg.norm(self.coef_ - coef_new) / np.linalg.norm(coef_new)
c:\Users\athar\anaconda4\Lib\site-packages\pygam\pygam.py:806: RuntimeWarning: invalid value encountered in scalar divide
  diff = np.linalg.norm(self.coef_ - coef_new) / np.linalg.norm(coef_new)
c:\Users\athar\anaconda4\Lib\site-packages\pygam\pygam.py:800: RuntimeWarning: divide by zero encountered in reciprocal
  np.fill_diagonal(Dinv, d**-1)  # invert the singular values
c:\Users\athar\anaconda4\Lib\site-packages\pygam\pygam.py:800: RuntimeWarning: overflow encountered in reciprocal
  np.fill_diagonal(Dinv, d**-1)  # invert the singular values


KeyboardInterrupt: 